In [1]:
param_x_api_key = '2BeBXg69E53RaoUFhA7hhf17jCg1NkwHoJz3FxmJPNS3bedbpHcb'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                data = '''{"getAutoCareSearchResults":
                {"baseVehicleRegionId":2,
                 "searchQuery":"MASS AIRFLOW SENSOR",
                 "page":''' + str(input_.loc[a, 'Page']) + ''',
                 "perPage":100,
                 "categories":[],
                 "padbAttributes":[],
                 "partTypeIds":[5128],
                 "brandFacets":{"enabled":true},
                 "includeParts":true,
                 "padbAttributeFacets":{"enabled":true},
                 "partTypeFacets":{"enabled":true}}}'''

                headers = UA.get_User_Agent_Requests()
                headers['X-Api-Key'] = param_x_api_key

                resp = requests.post('https://webservice.opticatonline.com/autocare/v1/services/Catalog.jsonEndpoint',
                                     data=data,
                                     headers=headers,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                list_part = json_data['parts']
                
                # = = = = = = = = = = = = = = =
                
                list_part_number = [part['piesItem']['partNumber'].strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                list_brand_id = [part['piesItem']['brandCode'].strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
                                
                list_brand = [part['piesItem']['brandName'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_manufacturer_website = [part['piesItem']['brandAdditionalInfo']['websiteURL'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_gtin_ean = []
                for part in list_part:
                    gtin_ean = ''
                    if 'itemLevelGTIN' in part['piesItem']:
                        gtin_ean = part['piesItem']['itemLevelGTIN']['gtin'].strip()
                    list_gtin_ean.append(gtin_ean)
                
                # = = = = = = = = = = = = = = =
                
                list_type_of_part = [part['piesItem']['categoryName'].strip() + ' > ' + part['piesItem']['subCategoryName'].strip() + ' > ' + part['piesItem']['partTypeName'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_number_of_parts = [str(part['piesItem']['itemQuantitySize']['value']) + ' ' + part['piesItem']['itemQuantitySize']['uomName'].strip() if 'itemQuantitySize' in part['piesItem'] else '' for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_quantity_per_application = [str(part['piesItem']['quantityPerApplication']['value']) + ' ' + part['piesItem']['quantityPerApplication']['uomName'].strip() if 'quantityPerApplication' in part['piesItem'] else '' for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_minimum_order = []
                for part in list_part:
                    minimum_order = ''
                    if 'minimumOrderQuantity' in list_part[0]['piesItem']:
                        minimum_order = str(list_part[0]['piesItem']['minimumOrderQuantity']['value']) + ' ' + list_part[0]['piesItem']['minimumOrderQuantity']['uomName'].strip()
                    list_minimum_order.append(minimum_order)

                # = = = = = = = = = = = = = = =
                
                list_list_description = [part['piesItem']['descriptions'] for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                list_list_extended_information = [part['piesItem']['extendedInformation'] for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_list_product_attribute = [part['piesItem']['productAttributes'] for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                list_list_package = [part['piesItem']['packages'] for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                list_oe = [';'.join([oe['partNumber'].strip() for oe in part['piesItem']['partInterchanges']]) for part in list_part]

                # = = = = = = = = = = = = = = =

                list_src = []
                for part in list_part:
                    src = ''
                    list_src_ = part['piesItem']['digitalAssets']
                    for src_ in list_src_:
                        if 'imageURL3200' in src_:
                            src = src_['imageURL3200'].strip()
                            break
                    list_src.append(src)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Page': int(input_.loc[a, 'Page']),
                                        '序号': [i+1 for i in range(len(list_part))],
                                        'Part_Number': list_part_number,
                                        'Brand_Id': list_brand_id,
                                        'Brand': list_brand,
                                        'Manufacturer_Website': list_manufacturer_website,
                                        'GTIN_EAN': list_gtin_ean,
                                        'Type_Of_Part': list_type_of_part,
                                        'Number_Of_Parts': list_number_of_parts,
                                        'Qantity_Per_Aplication': list_quantity_per_application,
                                        'Minimum_Order': list_minimum_order,
                                        'Vehicle': '',
                                        'OE': list_oe,
                                        'Pic': '',
                                        'Url': ['https://web.tecalliance.net/walkerproducts/qa/parts/' + brand_id + '/' + part_number + '/ap-detail' for brand_id, part_number in zip(list_brand_id, list_part_number)],
                                        'Src': list_src,
                                        'Vehicle_Page': '',
                                        'JOIN_PNBI': [part_number + ';' + brand_id for part_number, brand_id in zip(list_part_number, list_brand_id)]})

                for j in range(len(df_temp)):
                    list_description = list_list_description[j]
                    for description in list_description:
                        df_temp.loc[j, 'Description_' + description['descriptionTypeName'].strip() + ' (' + description['languageCode'].strip() + ')'] = description['value'].strip()

                    list_extended_information = list_list_extended_information[j]
                    for extended_information in list_extended_information:
                        df_temp.loc[j, 'Additional_' + extended_information['expiName'].strip()] = extended_information['formattedValue'].strip()

                    list_product_attribute = list_list_product_attribute[j]
                    for product_attribute in list_product_attribute:
                        if 'customAttributeName' in product_attribute:
                            df_temp.loc[j, 'Attribute_' + product_attribute['customAttributeName'].strip()] = product_attribute['value'].strip()
                        else:
                            df_temp.loc[j, 'Attribute_' + product_attribute['padbAttributeName'].strip()] = product_attribute['value'].strip() + ' ' + product_attribute['uomCode'].strip() if 'uomCode' in product_attribute else product_attribute['value'].strip()

                    list_package = list_list_package[j]
                    for i in range(len(list_package)):
                        for name in list_package[i]:
                            if name == 'dimensions':
                                df_temp.loc[j, 'Package_' + str(i+1) + '_height'] = str(list_package[i][name]['height']) + ' ' + list_package[i][name]['uomName']
                                df_temp.loc[j, 'Package_' + str(i+1) + '_width'] = str(list_package[i][name]['width']) + ' ' + list_package[i][name]['uomName']
                                df_temp.loc[j, 'Package_' + str(i+1) + '_length'] = str(list_package[i][name]['length']) + ' ' + list_package[i][name]['uomName']
                            elif name == 'weights':
                                df_temp.loc[j, 'Package_' + str(i+1) + '_weight'] = str(list_package[i][name]['weight']) + ' ' + list_package[i][name]['uomName']
                            elif list_package[i][name] == []:
                                pass
                            elif type(list_package[i][name]) == dict:
                                df_temp.loc[j, 'Package_' + str(i+1) + '_' + name.strip()] = str(list_package[i][name]['value']) + ' ' + list_package[i][name]['uomName']
                            else:
                                df_temp.loc[j, 'Package_' + str(i+1) + '_' + name.strip()] = list_package[i][name]

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Page': int(input_.loc[a, 'Page'])}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Page'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Page', '序号'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Page'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：13

[ok] - 6
[尝试次数：1] - [剩余数量：6] - [当前时间：20:11:32]

[ok] - 1
[尝试次数：1] - [剩余数量：5] - [当前时间：20:11:33]

[ok] - 3
[尝试次数：1] - [剩余数量：4] - [当前时间：20:11:33]

[ok] - 2
[尝试次数：1] - [剩余数量：3] - [当前时间：20:11:34]

[ok] - 7
[尝试次数：1] - [剩余数量：2] - [当前时间：20:11:34]

[ok] - 8
[尝试次数：1] - [剩余数量：1] - [当前时间：20:11:34]

[ok] - 5
[尝试次数：1] - [剩余数量：0] - [当前时间：20:11:34]

[ok] - 4
[尝试次数：1] - [剩余数量：0] - [当前时间：20:11:36]

[ok] - 10
[尝试次数：1] - [剩余数量：0] - [当前时间：20:11:36]

[ok] - 9
[尝试次数：1] - [剩余数量：0] - [当前时间：20:11:37]

[ok] - 11
[尝试次数：1] - [剩余数量：0] - [当前时间：20:11:38]

[ok] - 12
[尝试次数：1] - [剩余数量：0] - [当前时间：20:11:38]

[ok] - 13
[尝试次数：1] - [剩余数量：0] - [当前时间：20:11:39]

输出ing...

Done ~
